In [2]:
import tensorflow as tf
import numpy as np
import scipy.stats
import convolution as c
import os
from dataset import Dataset, chunks
from alexnet import AlexNet
from datetime import datetime

#Dataset setup
batch_size = 50
data = Dataset("dataset/flowers102", batch_size)
test_set = list(chunks(data.test[0], 50)), list(chunks(data.test[1],50))
test_size = len(data.test[0])
train_batches_per_epoch = int(np.ceil(len(data.train[0])/batch_size))

In [3]:
#Parameter setup
lr = 0.001
lr_decay = 0.1
lr_step = 203
epochs = 406
dropout_rate = 0.5
display_step = 1
show_step = display_step

#Higher learning rate layers
layers = ['fc8',]

#Number of classes
num_classes = 102

In [4]:
#Logging setup
lpath = "/tmp/flowers/logs"
mpath = "/tmp/flowers/models/"
log = "/tmp/flowers/{}-{}.log"
writer = tf.summary.FileWriter(lpath)

if not os.path.exists(mpath):
    os.makedirs(mpath)
    
def load_log(file):
    try:
        with open(file, 'r') as f:
            print(f.read(), end='')
    except:
        open(file, 'w').close()
        
def save_log(file, data):
    with open(file, 'a') as f:
        f.write(data+"\n")
        print(data)

In [8]:
def save_summaries(sess, data, train_summary, test_summary, batch, acc, iteration):
        s = train_summary.eval(session=sess, feed_dict={data[0]: batch[0],
                                                        data[1]: batch[1],
                                                        data[2]: 1})
        writer.add_summary(s, iteration)
        s = test_summary.eval(session=sess, feed_dict={data[3]: acc})
        writer.add_summary(s, iteration)
        writer.flush()

def run(scope, rate=1, conv=c.conv2d):
        
    tf.reset_default_graph()

    global lr
    logfile = log.format(scope, rate)
    load_log(logfile)
    
    # Placeholder for input/output and parameters
    x = tf.placeholder(tf.float32, [None, 227, 227, 3])
    y = tf.placeholder(tf.float32, [None, num_classes])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate1 = tf.placeholder(tf.float32)
    learning_rate2 = tf.placeholder(tf.float32)
    scalar = tf.placeholder(tf.float32) # placeholder for accuracy summary

    # Initialize model
    model = AlexNet(x, keep_prob, num_classes, layers, conv, rate=rate)

    # Link variable to model output
    score = model.fc8

    # List of trainable variables of the layers we want to train
    var_list1 = [v for v in tf.trainable_variables() if v.name.split('/')[0] in layers]
    var_list2 = [v for v in tf.trainable_variables() if v.name.split('/')[0] not in layers]

    # Loss OP
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = y))  
    
    # Iteration var
    global_step = tf.Variable(0)

    # Train op
    # Get gradients of all trainable variables
    gradients1 = tf.gradients(loss, var_list1)
    gradients1 = list(zip(gradients1, var_list1))
    gradients2 = tf.gradients(loss, var_list2)
    gradients2 = list(zip(gradients2, var_list2))

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer1 = tf.train.GradientDescentOptimizer(learning_rate1)
    optimizer2 = tf.train.GradientDescentOptimizer(learning_rate2)
    train_op1 = optimizer1.apply_gradients(grads_and_vars=gradients1, global_step=global_step)
    train_op2 = optimizer2.apply_gradients(grads_and_vars=gradients2)

    # Group up
    train_op = tf.group(train_op1, train_op2)

    correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Add the loss to summary
    with tf.name_scope(scope):
        with tf.name_scope(str(rate)):
            with tf.name_scope("test"):
                test = tf.summary.scalar('accuracy', scalar)
            with tf.name_scope("train"):
                train1 = tf.summary.scalar('accuracy', accuracy)
                train2 = tf.summary.scalar('loss', loss)
                train = tf.summary.merge([train1, train2])
            
    # Initializing
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.as_default()
    saver = tf.train.Saver()
    
    try:
        saver.restore(sess, "{}{}-{}.ckpt".format(mpath, scope, rate))
    except:
        sess.run(tf.global_variables_initializer())
        model.load_initial_weights(sess)
        save_log(logfile, "Running model: {} FM Rate: {}".format(scope, rate))
    
    #current epoch
    starting = int(global_step.eval(session=sess)/123)

    # Loop over number of epochs
    for epoch in range(starting, epochs):
        
        if epoch % lr_step == 0 and epoch != 0:
            lr *= lr_decay

        for _ in range(train_batches_per_epoch):
            # Get a batch of images and labels
            batch_xs, batch_ys = data.next_batch()

            # And run the training op
            sess.run(train_op, feed_dict={x: batch_xs, 
                                          y: batch_ys, 
                                          keep_prob: dropout_rate,
                                          learning_rate1: lr*10,
                                          learning_rate2: lr})
        
        if (epoch+1) % display_step == 0:
            
            acc = []
            for i in range(len(test_set[0])):
                acc.append(sess.run(accuracy, feed_dict={x: test_set[0][i], 
                                                         y: test_set[1][i], 
                                                         keep_prob: 1.}))
            acc = np.mean(acc)
            
            save_summaries(sess, [x, y, keep_prob, scalar], train, test, [batch_xs, batch_ys], acc, epoch+1)
            if (epoch+1) % show_step == 0:
                save_log(logfile, "Epoch: {} Test Accuracy: {:.4f}".format((epoch+1), acc))

        data.shuffle()
        saver.save(sess, "{}{}-{}.ckpt".format(mpath, scope, rate))
    sess.close()

In [10]:
rates = [1, 1.4, 1.6, 1.8, 2]

run("basic")
#for rate in rates:
#run("conv_bin_weights_vector", conv=c.conv_bin_weights_vector, rate=1)
run("conv_bin_weights_scalar", conv=c.conv_bin_weights_scalar, rate=1)


Running model: basic FM Rate: 1
Epoch: 1 Test Accuracy: 0.0381
Epoch: 2 Test Accuracy: 0.2095
Epoch: 3 Test Accuracy: 0.4205
Epoch: 4 Test Accuracy: 0.5810
Epoch: 5 Test Accuracy: 0.6533
Epoch: 6 Test Accuracy: 0.7181
Epoch: 7 Test Accuracy: 0.7238
Epoch: 8 Test Accuracy: 0.7486
Epoch: 9 Test Accuracy: 0.7814
Epoch: 10 Test Accuracy: 0.7748
Epoch: 11 Test Accuracy: 0.7914
Epoch: 12 Test Accuracy: 0.8010
Epoch: 13 Test Accuracy: 0.7900
Epoch: 14 Test Accuracy: 0.7914
Epoch: 15 Test Accuracy: 0.8143
Epoch: 16 Test Accuracy: 0.1390
Epoch: 17 Test Accuracy: 0.7571
Epoch: 18 Test Accuracy: 0.7748
Epoch: 19 Test Accuracy: 0.7933
Epoch: 20 Test Accuracy: 0.8005
Epoch: 21 Test Accuracy: 0.8067
Epoch: 22 Test Accuracy: 0.8110
Epoch: 23 Test Accuracy: 0.8057
Epoch: 24 Test Accuracy: 0.8157
Epoch: 25 Test Accuracy: 0.8224
Epoch: 26 Test Accuracy: 0.8148
Epoch: 27 Test Accuracy: 0.8281
Epoch: 28 Test Accuracy: 0.8267
Epoch: 29 Test Accuracy: 0.8233
Epoch: 30 Test Accuracy: 0.8290
Epoch: 31 Test Ac